In [1]:
#include <TFile.h>
#include <TTree.h>
#include <TString.h>
#include <TH1D.h>
#include <TCanvas.h>
#include <TSystem.h>
#include <iostream>

In [2]:
#ifndef _COMPARE_MC_REAL_HH_
#define _COMPARE_MC_REAL_HH_

class baseHist
{
public:
    TH1D* hMass;
    TH1D* hpT;
    TH1D* hxF;
    TH1D* hx1;
    TH1D* hx2;
    baseHist(TString oname);
    virtual ~baseHist();
    void fillHist(TTree* tree);
};


class compareHist
{
    baseHist* hist_mc;
    baseHist* hist_real;
public:
    compareHist(TString name_mc, TString name_real);
    virtual ~compareHist();
    void compare(TTree* mc_tree, TTree* real_tree);
};

#endif /*_COMPARE_MC_REAL_HH_*/

In [3]:
baseHist::baseHist(TString oname)
{
    TString hmass_name = Form("hmass_%s", oname.Data());
    TString hpT_name = Form("hpT_%s", oname.Data());
    TString hxF_name = Form("hxF_%s", oname.Data());
    TString hxT_name = Form("hxT_%s", oname.Data());
    TString hxB_name = Form("hxB_%s", oname.Data());

    hMass = new TH1D(hmass_name.Data(), "; mass [GeV]; Yield", 20, 4.5, 9.);
    hpT = new TH1D(hpT_name.Data(), "; pT [GeV]; Yield", 20, 0.0, 2.5);
    hxF = new TH1D(hxF_name.Data(), "; xF; Yield", 20, -0.1, 0.95);
    hx1 = new TH1D(hxT_name.Data(), "; xT; Yield", 20, 0.3, 1.0);
    hx2 = new TH1D(hxB_name.Data(), "; xB; Yield", 20, 0.1, 0.6);

    hMass->Sumw2();
    hpT->Sumw2();
    hxF->Sumw2();
    hx1->Sumw2();
    hx2->Sumw2();
}

In [4]:
baseHist::~baseHist()
{
    delete hMass;
    delete hpT;
    delete hxF;
    delete hx1;
    delete hx2;
}

In [5]:
void baseHist::fillHist(TTree* tree)
{
    double mass, pT, xF, x1, x2, weight;

    tree->SetBranchAddress("mass", &mass);
    tree->SetBranchAddress("pT", &pT);
    tree->SetBranchAddress("xF", &xF);
    tree->SetBranchAddress("x1", &x1);
    tree->SetBranchAddress("x2", &x2);
    tree->SetBranchAddress("weight", &weight);

    for(int ii = 0; ii < tree->GetEntries(); ii++)
    {
        tree->GetEntry(ii);
        hMass->Fill(mass, weight);
        hpT->Fill(pT, weight);
        hxF->Fill(xF, weight);
        hx1->Fill(x1, weight);
        hx2->Fill(x2, weight);
    }
}

In [6]:
compareHist::compareHist(TString name_mc, TString name_real)
{
    hist_mc = new baseHist(name_mc);
    hist_real = new baseHist(name_real);

    gSystem->mkdir("imgs/compare_mc_real", 1);
}

In [7]:
compareHist::~compareHist()
{
    delete hist_mc;
    delete hist_real;
}

In [8]:
void compareHist::compare(TTree* mc_tree, TTree* real_tree)
{
    hist_mc->fillHist(mc_tree);
    hist_real->fillHist(real_tree);

    TH1D* hist_mass_real = (TH1D*)hist_real->hMass->Clone();
    hist_mass_real->Divide(hist_mc->hMass);
    hist_mass_real->SetName("ratio_hmass");

    TH1D* hist_pT_real = (TH1D*)hist_real->hpT->Clone();
    hist_pT_real->Divide(hist_mc->hpT);
    hist_pT_real->SetName("ratio_hpT");

    TH1D* hist_xF_real = (TH1D*)hist_real->hxF->Clone();
    hist_xF_real->Divide(hist_mc->hxF);
    hist_xF_real->SetName("ratio_hxF");

    TH1D* hist_x1_real = (TH1D*)hist_real->hx1->Clone();
    hist_x1_real->Divide(hist_mc->hx1);
    hist_x1_real->SetName("ratio_hxT");

    TH1D* hist_x2_real = (TH1D*)hist_real->hx2->Clone();
    hist_x2_real->Divide(hist_mc->hx2);
    hist_x2_real->SetName("ratio_hxB");

    TFile* outfile = new TFile("compare.root", "RECREATE");

    double mass_area = hist_real->hMass->Integral();
    hist_real->hMass->Scale(1./mass_area);
    hist_mc->hMass->Scale(1./mass_area);

    double pT_area = hist_real->hpT->Integral();
    hist_real->hpT->Scale(1./pT_area);
    hist_mc->hpT->Scale(1./pT_area);

    double xF_area = hist_real->hxF->Integral();
    hist_real->hxF->Scale(1./xF_area);
    hist_mc->hxF->Scale(1./xF_area);

    double x1_area = hist_real->hx1->Integral();
    hist_real->hx1->Scale(1./x1_area);
    hist_mc->hx1->Scale(1./x1_area);

    double x2_area = hist_real->hx2->Integral();
    hist_real->hx2->Scale(1./x2_area);
    hist_mc->hx2->Scale(1./x2_area);

    hist_real->hMass->Write();
    hist_real->hpT->Write();
    hist_real->hxF->Write();
    hist_real->hx1->Write();
    hist_real->hx2->Write();

    hist_mc->hMass->Write();
    hist_mc->hpT->Write();
    hist_mc->hxF->Write();
    hist_mc->hx1->Write();
    hist_mc->hx2->Write();

    hist_mass_real->Write();
    hist_pT_real->Write();
    hist_xF_real->Write();
    hist_x1_real->Write();
    hist_x2_real->Write();
    
    outfile->Close();

    gSystem->Exec("python compare_hist_plots.py");
}

In [9]:
TFile* infile = TFile::Open("reweight_data.root", "read");

TTree* train_mc = (TTree*)infile->Get("train_mc");
TTree* tree_real = (TTree*)infile->Get("train_tree");
TTree* test_mc = (TTree*)infile->Get("test_mc");

In [10]:
compareHist* ch = new compareHist("mc", "real");

In [11]:
ch->compare(train_mc, tree_real);